In [1]:
import json
import math
import random
import os  # os 모듈 추가
from datetime import datetime, timedelta

# --- 설정 ---
START_TIME = datetime(2025, 11, 21, 15, 0, 0)
DURATION_HOURS = 1
TAG1_POS = (0, 0)

# [수정된 부분] 파일을 바탕화면(Desktop) 경로에 저장하도록 강제 지정
desktop_path = os.path.expanduser("~/Desktop")
FILENAME = os.path.join(desktop_path, "collision_data_1hour.json")

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def get_message(distance, is_approaching):
    if distance <= 1.5:
        return "Critical: Stop immediately"
    elif is_approaching:
        return "Caution: Vehicle approaching worker"
    else:
        return "Caution: Vehicle retreating"

def generate_event(start_timestamp):
    events = []
    angle = random.uniform(0, 2 * math.pi)
    min_dist = random.uniform(0.5, 2.5)
    current_dist = 3.1
    step_size = random.uniform(0.2, 0.4)
    current_time = start_timestamp
    
    # 1. 접근
    while current_dist > min_dist:
        current_dist -= step_size
        if current_dist < min_dist: current_dist = min_dist
        x = current_dist * math.cos(angle)
        y = current_dist * math.sin(angle)
        
        if current_dist <= 3.0:
            events.append({
                "timestamp": current_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z",
                "eventType": "COLLISION_WARNING",
                "sourceTag": "Tag2",
                "targetTag": "Tag1",
                "distance": round(current_dist, 2),
                "x": round(x, 2),
                "y": round(y, 2),
                "message": get_message(current_dist, True)
            })
        current_time += timedelta(seconds=0.5)

    # 잠시 멈춤
    pause_steps = random.randint(0, 4)
    for _ in range(pause_steps):
        current_time += timedelta(seconds=0.5)
        events.append({
            "timestamp": current_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z",
            "eventType": "COLLISION_WARNING",
            "sourceTag": "Tag2",
            "targetTag": "Tag1",
            "distance": round(current_dist, 2),
            "x": round(current_dist * math.cos(angle), 2),
            "y": round(current_dist * math.sin(angle), 2),
            "message": get_message(current_dist, True)
        })

    # 2. 후퇴
    while current_dist < 3.1:
        current_dist += step_size
        x = current_dist * math.cos(angle)
        y = current_dist * math.sin(angle)
        
        if current_dist <= 3.0:
            events.append({
                "timestamp": current_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z",
                "eventType": "COLLISION_WARNING",
                "sourceTag": "Tag2",
                "targetTag": "Tag1",
                "distance": round(current_dist, 2),
                "x": round(x, 2),
                "y": round(y, 2),
                "message": get_message(current_dist, False)
            })
        current_time += timedelta(seconds=0.5)
        
    return events, current_time

def main():
    full_log = []
    current_sim_time = START_TIME
    end_time = START_TIME + timedelta(hours=DURATION_HOURS)
    incident_count = 0
    
    print(f"Generating data from {START_TIME} to {end_time}...")
    
    while current_sim_time < end_time:
        safe_duration = random.randint(120, 480) 
        current_sim_time += timedelta(seconds=safe_duration)
        
        if current_sim_time >= end_time:
            break
            
        new_events, end_event_time = generate_event(current_sim_time)
        full_log.extend(new_events)
        current_sim_time = end_event_time
        incident_count += 1

    with open(FILENAME, 'w', encoding='utf-8') as f:
        json.dump(full_log, f, indent=2, ensure_ascii=False)
        
    print(f"✅ 완료! 총 {incident_count}번의 위험 상황 발생.")
    print(f"📂 저장된 파일 위치: {FILENAME}")

if __name__ == "__main__":
    main()

Generating data from 2025-11-21 15:00:00 to 2025-11-21 16:00:00...
✅ 완료! 총 11번의 위험 상황 발생.
📂 저장된 파일 위치: /Users/hanjeonghui/Desktop/collision_data_1hour.json
